In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 28.7 MB/s 
     |████████████████████████████████| 7.6 MB 50.7 MB/s 
     |████████████████████████████████| 163 kB 71.9 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import platform
%matplotlib inline
import matplotlib
import tensorflow as tf
from sklearn.metrics import roc_curve,confusion_matrix,auc,  roc_auc_score
BATCH_SIZE=6 # 커널이 계속 죽는다.. (메모리 초과...;) -> 배치 사이즈가 6보다 크면 메모리 초과 발생;
MAX_SEQ_LEN = 512

import os # GPU 설정
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0"; 

import warnings # warning message ignore
warnings.filterwarnings(action='ignore')

# install transformers

import transformers

# model import
from transformers import DistilBertConfig, TFDistilBertForSequenceClassification, DistilBertTokenizer
from sklearn.model_selection import train_test_split

# 한글 폰트 설정
if platform.system() == 'Darwin': # mac
        plt.rc('font', family='AppleGothic') 
else: # window or linux
        plt.rc('font', family='NanumGothicCoding') 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#주최측 공식 데이터
# train_df = pd.read_csv('/content/gdrive/My Drive/kerc/data/tmp/df_train_extended.tsv',sep = '\t')
# train_labels_df = pd.read_csv('/content/gdrive/My Drive/kerc/data/train_labels.csv')
# test_df = pd.read_csv('/content/gdrive/My Drive/kerc/data/tmp/df_public_test_extended.tsv', sep = '\t')

#lee 데이터
train_df = pd.read_csv('/content/gdrive/My Drive/kerc/data/lee/df_train_extended.tsv',sep = '\t')
train_labels_df = pd.read_csv('/content/gdrive/My Drive/kerc/data/lee/train_labels.csv')
test_df = pd.read_csv('/content/gdrive/My Drive/kerc/data/lee/df_public_test_extended.tsv', sep = '\t')

In [ ]:
train_df = pd.merge(train_df, train_labels_df, on = 'sentence_id', how = 'left')

In [ ]:
train_df = train_df.replace(np.nan, '', regex=True)
test_df = test_df.replace(np.nan, '', regex=True)
train_df["sentence"]  = train_df["sentence"].str.replace("\s+", " ", regex=True)+" "+train_df["context"].str.replace("\s+", " ", regex=True)+" "+train_df["target_speaker_ctx"].str.replace("\s+", " ", regex=True)+" "+train_df["other_speaker_ctx"].str.replace("\s+", " ", regex=True)+" "+train_df["scene_sents"].str.replace("\s+", " ", regex=True)
test_df["sentence"]  = test_df["sentence"].str.replace("\s+", " ", regex=True)+" "+test_df["context"].str.replace("\s+", " ", regex=True)+" "+test_df["target_speaker_ctx"].str.replace("\s+", " ", regex=True)+" "+test_df["other_speaker_ctx"].str.replace("\s+", " ", regex=True)+" "+test_df["scene_sents"].str.replace("\s+", " ", regex=True)

In [ ]:
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE

# X, y = make_classification(n_samples=10000, n_features=5, weights=[0.99], flip_y=0)
# print(Counter(y))
# print(X)
# print(y)


sentence = train_df["sentence"].values.tolist()
label = train_df["label"].values.tolist()
print(label)
sentence_arr = []
for sent in sentence:
  sentence_arr.append([sent])

# X_under, y_under = RandomUnderSampler(random_state=0).fit_resample(sentence_arr, label)
# print(Counter(y_under))

X_under, y_under = RandomOverSampler(random_state=0).fit_resample(sentence_arr, label)
print(Counter(y_under))

X_under_arr = []
for X in X_under:
  X_under_arr.extend(X)

#print(X_under_arr)
train_df_under = pd.DataFrame()
train_df_under["sentence"] = X_under_arr
train_df_under["label"] = y_under

['dysphoria', 'dysphoria', 'neutral', 'dysphoria', 'euphoria', 'neutral', 'euphoria', 'euphoria', 'euphoria', 'dysphoria', 'dysphoria', 'dysphoria', 'neutral', 'neutral', 'dysphoria', 'dysphoria', 'neutral', 'euphoria', 'euphoria', 'euphoria', 'euphoria', 'euphoria', 'euphoria', 'neutral', 'euphoria', 'euphoria', 'euphoria', 'neutral', 'euphoria', 'euphoria', 'euphoria', 'euphoria', 'euphoria', 'dysphoria', 'dysphoria', 'dysphoria', 'euphoria', 'neutral', 'dysphoria', 'dysphoria', 'dysphoria', 'neutral', 'neutral', 'dysphoria', 'dysphoria', 'neutral', 'neutral', 'neutral', 'dysphoria', 'dysphoria', 'dysphoria', 'dysphoria', 'neutral', 'neutral', 'dysphoria', 'neutral', 'dysphoria', 'dysphoria', 'dysphoria', 'dysphoria', 'dysphoria', 'dysphoria', 'dysphoria', 'dysphoria', 'dysphoria', 'dysphoria', 'dysphoria', 'dysphoria', 'dysphoria', 'dysphoria', 'dysphoria', 'dysphoria', 'dysphoria', 'euphoria', 'euphoria', 'euphoria', 'euphoria', 'euphoria', 'euphoria', 'euphoria', 'euphoria', 'euph

In [ ]:
MODEL_NAME = 'distilbert-base-multilingual-cased'  # DOES NOT WORK
tokenizer = DistilBertTokenizer.from_pretrained(MODEL_NAME)

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466 [00:00<?, ?B/s]

In [ ]:
model = TFDistilBertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels = 3)

Downloading:   0%|          | 0.00/911M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-multilingual-cased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'activation_13', 'vocab_transform', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_19']
You should probably TRAIN this model on a down-stream ta

In [ ]:
from sklearn.preprocessing import LabelEncoder

# X = train_df_under["sentence"].to_list()
# labelencoder = LabelEncoder()
# train_df_under["label"] = labelencoder.fit_transform(train_df_under["label"])
# y = train_df_under["label"].to_list()

X = train_df["sentence"].to_list()
labelencoder = LabelEncoder()
train_df["label"] = labelencoder.fit_transform(train_df["label"])
y = train_df["label"].to_list()

In [ ]:
X_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
train_encoding = tokenizer(X_train,
                            truncation=True, 
                            padding=True)
val_encoding = tokenizer(x_val,
                            truncation=True, 
                            padding=True)

In [ ]:
# 학습 데이터셋으로 변환
# train - validation batch size 반드시 맞춰서 입력해야함
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encoding),
    y_train
)).shuffle(len(X_train)).repeat().batch(BATCH_SIZE)

# validation-set
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encoding),
    y_val
)).batch(BATCH_SIZE)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=opt, loss=loss, metrics=[metric])
history = model.fit(train_dataset, epochs=3, steps_per_epoch=len(X_train)//BATCH_SIZE, # Early Stopping으로 학습 최대점은 에폭 4회일 때임
                    validation_data=val_dataset, validation_steps=len(x_val)//BATCH_SIZE, verbose=1)

Epoch 1/3
7113/7113 [==============================] - 3069s 430ms/step - loss: 0.3183 - accuracy: 0.8799 - val_loss: 0.2646 - val_accuracy: 0.8925
Epoch 2/3
7113/7113 [==============================] - 3064s 431ms/step - loss: 0.2222 - accuracy: 0.9182 - val_loss: 0.2350 - val_accuracy: 0.9100
Epoch 3/3
7113/7113 [==============================] - 3064s 431ms/step - loss: 0.1849 - accuracy: 0.9336 - val_loss: 0.2576 - val_accuracy: 0.9055


In [ ]:
X_test = test_df["sentence"].to_list()
test_encoding = tokenizer(X_test,
                            truncation=True, 
                            padding=True)
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encoding),
)).batch(BATCH_SIZE)
prediction = model.predict(test_dataset, verbose=1)

428/428 [==============================] - 56s 128ms/step


In [ ]:
answer = np.array([y.argmax() for y in prediction["logits"]])

submission_df = pd.DataFrame()
submission_df["Id"] = test_df['sentence_id']
submission_df["Predicted"] = labelencoder.inverse_transform(answer)
submission_df.to_csv(f'/content/gdrive/MyDrive/kerc/logs/submission_deep_lee.csv', index=False)
print(f"Saved 'logs/submission_deep_lee.csv")



Saved 'logs/submission_deep_lee.csv


In [ ]:
print(submission_df["Predicted"].value_counts())

dysphoria    1625
euphoria      623
neutral       318
Name: Predicted, dtype: int64


dysphoria    2084
euphoria      334
neutral       148 // 0.68394

---
dysphoria    1346
euphoria      808
neutral       412 // 0.63133
